In [1]:
import numpy as np
import pandas as pd
import os
import re

In [24]:
def load_csv(filename, skip_rows):
    df = pd.read_csv(filename, skiprows=skip_rows, header=0)
    df.drop(df.columns[0], axis=1, inplace=True)
    return df


def calculate_avg(df):  # prints dataframe to check
    df_even = df.iloc[:, 0::6]
    display(df_even)
    even_row_avg = df_even.iloc[-1].mean()
    print('AVG Cumulative Profit of retailer 0:', even_row_avg)

    df_odd = df.iloc[:, 3::6]
    display(df_odd)
    odd_row_avg = df_odd.iloc[-1].mean()
    print('AVG Cumulative Profit of retailer 1:', odd_row_avg)


def calculate_payoffs(df):
    retailer_0 = df.iloc[:, 0::6]
    retailer_0_payoff = round(retailer_0.iloc[-1].mean(), 2)

    retailer_1 = df.iloc[:, 3::6]
    retailer_1_payoff = round(retailer_1.iloc[-1].mean(), 2)
    return (retailer_0_payoff, retailer_1_payoff)


def get_column_dominant(lst):
    payoffs = [eval(s)[1] for s in lst]
    max_payoff = max(payoffs)
    dominant_index = payoffs.index(max_payoff)
    opponent_payoff = lst[dominant_index].strip('[').split(',')[0]
    dominant_str = f'[{opponent_payoff}, {max_payoff}*]'
    return (dominant_index, dominant_str)


def get_row_dominant(lst):
    payoffs = [float(s.strip('[').split(',')[0]) for s in lst]
    max_payoff = max(payoffs)
    dominant_index = payoffs.index(max_payoff)
    opponent_payoff = lst[dominant_index].strip(']').split(', ')[1]
    dominnant_str = f'[{max_payoff}*, {opponent_payoff}]'
    return (dominant_index, dominnant_str)


def construct_payoff_table(file_str):
    columns = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6']
    payoffs = [['x' for y in range(6)] for x in range(6)]

    # generate payoff table
    for i in range(1, len(payoffs) + 1):
        for j in range(1, len(payoffs) + 1):
            filename = f'./experiment_data/S{i}_S{j}{file_str}.csv'
            if os.path.isfile(filename):
                df = load_csv(filename, 34)
                payoff = calculate_payoffs(df)
                payoff_str = f'[{payoff[0]}, {payoff[1]}]'
                payoffs[i - 1][j - 1] = payoff_str

    # mark dominant strategies in retailer 1's perspective
    for i in range(0, len(payoffs)):
        cur_row = payoffs[i]
        (dominant_index, dominant_str) = get_column_dominant(cur_row)
        payoffs[i][dominant_index] = dominant_str

    # mark dominant strategies in retailer 0's perspective
    for i in range(0, len(payoffs)):
        cur_column = [row[i] for row in payoffs]
        (dominant_index, dominant_str) = get_row_dominant(cur_column)
        payoffs[dominant_index][i] = dominant_str

    # convert to dataframe
    table = pd.DataFrame(payoffs, columns=columns)
    table = table.rename(index={0: 'S1', 1: 'S2', 2: 'S3', 3: 'S4', 4: 'S5', 5: 'S6'})
    display(table)

# Payoff table of price weight=2

In [25]:
construct_payoff_table('_price_2')

,S1,S2,S3,S4,S5,S6
S1,"[195018.75, 199574.35*]","[156512.85, 184980.85]","[152344.05, 151445.5]","[124127.05, 110202.0]","[126540.25, 121236.5]","[125433.8*, 162172.0]"
S2,"[253094.72*, 230575.55*]","[196914.05, 216158.3]","[194366.5, 142389.5]","[108463.7, 138837.75]","[114539.75, 113252.0]","[103210.05, 99662.5]"
S3,"[234221.0, 253022.4]","[236026.3*, 257017.95*]","[180784.25, 149406.0]","[126371.9, 118042.75]","[90292.55, 104933.5]","[86451.5, 98884.0]"
S4,"[185731.0, 249841.35*]","[183350.75, 216939.8]","[224823.38*, 239784.5]","[194708.0*, 226403.25]","[87107.0, 102313.0]","[87651.75, 60382.5]"
S5,"[231363.5, 224496.8*]","[192181.0, 152748.3]","[142405.5, 119375.25]","[100936.0, 100632.5]","[153714.5*, 126128.0]","[72023.0, 86369.5]"
S6,"[222494.0, 258300.6*]","[205614.5, 188707.15]","[160293.0, 127309.12]","[113901.0, 119167.75]","[114052.0, 127628.5]","[89500.5, 58066.0]"


# Payoff table of price weight=6

In [26]:
construct_payoff_table('_price_6')

,S1,S2,S3,S4,S5,S6
S1,"[210715.65, 132990.35*]","[160923.7*, 128416.2]","[141046.7, 117104.25]","[112553.85, 110935.5]","[121832.55, 104546.5]","[103962.3, 119801.0]"
S2,"[256517.48, 195554.15*]","[150891.65, 118761.5]","[148739.3, 116827.75]","[75196.85, 70558.0]","[96448.2, 78840.0]","[67154.5, 55146.5]"
S3,"[242176.75, 197992.65*]","[138390.25, 120093.15]","[114730.5, 91419.25]","[134091.5, 117971.0]","[85857.75, 84525.5]","[80167.75, 80754.0]"
S4,"[238284.25, 214720.25*]","[125558.0, 120248.4]","[167392.5*, 150284.25]","[164330.5*, 156036.0]","[155243.5*, 158404.5]","[105691.5*, 81768.5]"
S5,"[237910.5, 199162.15*]","[106854.0, 88980.2]","[124111.5, 115753.75]","[75442.0, 60428.0]","[89279.0, 83989.0]","[69023.5, 71877.0]"
S6,"[267071.0*, 218650.1*]","[126058.5, 133826.5]","[118846.5, 114537.5]","[60796.5, 56340.75]","[80340.0, 80933.5]","[93109.0, 107353.5]"


# Payoff table of price weight=10

In [27]:
construct_payoff_table('_price_10')

,S1,S2,S3,S4,S5,S6
S1,"[215655.3, 105533.1]","[156718.15*, 124377.2*]","[145578.7*, 111322.0]","[102122.9*, 89898.0]","[127767.1*, 94736.0]","[113596.1*, 103371.5]"
S2,"[229937.98, 162951.53*]","[112841.2, 78321.2]","[112578.15, 88997.75]","[46013.4, 50646.25]","[98552.75, 71599.5]","[74543.35, 53361.0]"
S3,"[239466.75, 160783.75*]","[89228.75, 66558.6]","[92956.0, 95276.75]","[59439.5, 46090.25]","[94534.0, 63712.5]","[76235.25, 75434.0]"
S4,"[248395.75*, 201163.6*]","[95675.25, 91256.2]","[118569.12, 103293.38]","[42682.5, 40310.25]","[90856.0, 76827.5]","[67475.25, 58088.0]"
S5,"[224352.0, 177530.7*]","[100782.5, 98525.55]","[107967.5, 95881.25]","[65983.0, 58772.75]","[101509.0, 99060.5]","[64640.5, 68198.5]"
S6,"[95830.5, 71869.5]","[102850.0, 91599.2*]","[109644.0, 91476.0]","[82797.5, 75275.5]","[71882.0, 80511.0]","[62041.5, 57542.5]"
